In [ ]:
!gdown --id 1kFyKea_Tk3mXyyTjyHuXdAstx9Aac03T

In [ ]:
!pip install git+https://github.com/JohnGiorgi/seq2rel.git

In [ ]:
!pip uninstall en-core-web-sm==3.7.1

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import json
from seq2rel import Seq2Rel
from seq2rel.common import util
from tqdm import tqdm

In [ ]:
nlp = spacy.load('en_core_web_sm')

MAX_SPLIT_LENGHT = 450

def split_text_words(text):
    cleaned_text = text
    tokens = nlp(cleaned_text)
    if len(tokens)<=MAX_SPLIT_LENGHT:
        return [tokens.text]
    splits = []

    start_index = 0
    end_index = MAX_SPLIT_LENGHT-1
    while end_index!=len(tokens)-1:
        if tokens[end_index].text not in ['.','?','!']:
            while tokens[end_index].text not in ['.','?','!']:
                end_index-=1
        split = tokens[start_index:end_index+1]
        splits.append(split.text.strip())
        start_index = end_index+1
        end_index = start_index+MAX_SPLIT_LENGHT-1 if (start_index+MAX_SPLIT_LENGHT-1<=len(tokens)-1) else len(tokens)-1
    return splits

In [ ]:
pretrained_model = "docred"
seq2rel = Seq2Rel(pretrained_model,cuda_device=0)

with open('data2.json') as f:
  data = json.load(f)


processed_data = []

for article in tqdm(data):
  text = article['text']
  splits = split_text_words(text)
  output = seq2rel(splits)
  extract_relations = util.extract_relations(output)
  for rel in extract_relations:
      for key, value in rel.items():
          for pair in value:
              processed_data.append({"subj": pair[0], "pred": key, "obj": pair[1]})


print(len(processed_data))
with open('extracted_triples.json','w') as f:
  json.dump(processed_data, f)

downloading: 100%|##########| 419M/419M [00:06<00:00, 67.1MiB/s]


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/allennlp/common/cached_transformers.py:81: UserWarning: You specified 'reinit_modules' in allennlp.common.cached_transformers.get(), but 'load_weights' is set to False, so 'reinit_modules' will be ignored.
  warnings.warn(
100%|██████████| 452/452 [2:22:17<00:00, 18.89s/it]

2570
